# Integrantes:

- Alejandro Arango Molina
- Lukas Guerra Escobar
- Juan Andrés Ortega Grajales
- Nilson Fernando Suarez Hernández

# Importación de modulos y lectura de las bases de datos

In [2]:
import pandas as pd
import numpy as np
import unidecode as udc
import warnings
import plotly.graph_objects as go 
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')

In [3]:
# lectura de las bases de datos como data-frame con la dirección directa de las bases de datos
cv_df = pd.read_csv('https://datosabiertos.bogota.gov.co/dataset/5e79c701-d02f-4220-a9ae-31c8def1cfab/resource/47c5207a-d597-4c9e-ad74-4fb980656bf8/download/osb_demografia-condicionesvida.csv', encoding='latin-1', delimiter=';', decimal=',')
vi_df = pd.read_csv('https://datosabiertos.bogota.gov.co/dataset/a1e1ef90-10c0-436f-a290-d1f7c1cf2242/resource/ab4eeb6e-e7c1-4ec1-b3e5-eedc655bc8d7/download/osb_v-intrafamiliar.csv', encoding='latin-1', delimiter=';', decimal=',', thousands='.')
sc_df = pd.read_csv('https://datosabiertos.bogota.gov.co/dataset/2b8464e3-3aca-4dcd-91a1-93dd06ddabbb/resource/f215cedd-46e0-44fe-ba4c-704afdc11a33/download/osb_saludmen-tsuicidiodesagregado.csv', encoding='latin-1', delimiter=';')

# Descripción de las Bases de Datos

Las bases de datos usadas fueron tomadas de la [pagina de datos abiertos](https://saludata.saludcapital.gov.co/osb/ "SALUDATA | Observatorio de Salud de Bogotá") de la Secretaria de Salud de Bogotá y corresponden a la siguiente informacion:

- `cv_df`: Encuesta relizada en 2017 en la ciudad sobre la percepción de condición de vida de los habitantes. Disponible en el siguiente [enlace](https://datosabiertos.bogota.gov.co/dataset/5e79c701-d02f-4220-a9ae-31c8def1cfab "Datos Abiertos Bogotá").
- `vi_df`: Datos de los casos de los diferentes tipos de violencia intrafamiliar en la ciudad por año desde 2012 hasta 2021. Disponible en el siguiente [enlace](https://datosabiertos.bogota.gov.co/dataset/a1e1ef90-10c0-436f-a290-d1f7c1cf2242 "Datos Abiertos Bogotá").
- `sc_df`: Datos de los casos de suicidios en la ciudad por año desde 2015 hasta 2021. Disponible en el siguiente [enlace](https://datosabiertos.bogota.gov.co/dataset/2b8464e3-3aca-4dcd-91a1-93dd06ddabbb "Datos Abiertos Bogotá").

# Tratamiento de valores, sintáxis y categorías

In [4]:
# 'Cleaning' en los nombres y tipos de cada columna para un mejor acceso
vi_df['Año'] = vi_df['Año'].str.extract(r'(\d{4})').astype(int)
vi_df.columns = [udc.unidecode(i) for i in vi_df.columns.str.lower().str.replace('.', '', regex=False).str.replace(' ', '_', regex=False)]
cv_df.columns = cv_df.columns.str.lower().str.replace(' ', '_')
sc_df.columns = [udc.unidecode(i) for i in sc_df.columns.str.lower().str.replace(' ', '_', regex=False)]

dfs = [cv_df, vi_df, sc_df]              # lista de data-frames

# Homologación de categorías
for i in dfs:
    if 'ano' in i.columns:
        i.rename(columns={'ano': 'year'}, inplace=True)
    if 'area' in i.columns:
        i.rename(columns={'area':'localidad'}, inplace=True)
        
# Normalización de categorías        
cv_df['localidad'] = cv_df['localidad'].map(udc.unidecode).str.lower().replace({'bogota d.c.': 'distrito'})
sc_df['localidad'] = sc_df['localidad'].map(udc.unidecode).str.lower()
vi_df['localidad'] = vi_df['localidad'].map(udc.unidecode).str.lower()

cv_df['condiciones_de_vida'] = cv_df['condiciones_de_vida'].str.lower()
vi_df['sexo'] = vi_df['sexo'].str.strip()

# Completar valores faltantes en los numeros de casos con 0
vi_df['no_casos'].fillna(0, inplace=True)                

# Completar valores faltantes en la tasa por 100mil habitantes con la operacion (no_casos/poblacion)*100000
vi_df['tasa_por_100000'].fillna((vi_df['no_casos']/vi_df['poblacion'])*100000, inplace=True)

# Adición de información a los Data Frames (Transformaciones)

In [5]:
# Columna adicional para identificar el grupo etáreo de cada grupo edad

# Diccionario con grupos de edad y respectivo grupo etáreo
dic = {'Niño': sc_df['grupo_de_edad'].unique()[:2], 'Joven': sc_df['grupo_de_edad'].unique()[2:5], 'Adulto': sc_df['grupo_de_edad'].unique()[5:12], 'Adulto Mayor': sc_df['grupo_de_edad'].unique()[12:-1]}

# Creación columna adicional vacía
sc_df['grupo_etario'] = np.nan

# Llenado columna vacía
for i in dic:
    sc_df.loc[sc_df['grupo_de_edad'].isin(dic[i]),'grupo_etario'] = i

In [6]:
# Columna adicional que lleve la población y la tasa de casos de suicidio por cada 100.000 habitantes

# Obtener datos de población por cada año, localidad y sexo
_1 = vi_df.groupby(['year', 'localidad', 'sexo'])[['year', 'localidad', 'sexo', 'poblacion']].sample(1) 

# Unión de la columna de población
sc_df = sc_df.merge(_1, on=['year', 'localidad', 'sexo'], how='inner') 

# Creación de la columna de tasa por cada 100.000 habitantes
sc_df['tasa_por_100000'] = (sc_df['casos']/sc_df['poblacion'])*100000 

# Filtrado para llenar solo la información diferenciada por grupo de edad
sc_df.loc[sc_df['grupo_de_edad'] != 'Todos los grupos', ['tasa_por_100000', 'poblacion']] = np.nan 

In [10]:
# Creacion de bodega de datos
cv_df.to_csv('bases_de_datos/condicion_final.csv', sep=';', index=False)
vi_df.to_csv('bases_de_datos/violencia_final.csv', sep=';', index=False)
sc_df.to_csv('bases_de_datos/suicidio_final.csv', sep=';', index=False)

In [9]:
import os


<module 'ntpath' from 'C:\\Users\\Lukas\\anaconda3\\lib\\ntpath.py'>


# Análisis Descriptivo

## Base Condiciones de vida

### 1. ¿Cuál es el Porcentaje promedio de condiciones de vida total en Bogotá?

In [ ]:
# Creación del dataset
df = cv_df.groupby(['condiciones_de_vida'], as_index=False)[['porcentaje']].mean()

# Creación de gráfica
fig = px.pie(df, values='porcentaje', names='condiciones_de_vida',
            title='<b>Porcentaje promedio de condiciones de vida total en Bogotá<b>',
            color_discrete_sequence=px.colors.qualitative.Safe, opacity=1, hover_data={'condiciones_de_vida':False, 'porcentaje':False})

# Agregar detalles a gráfica
fig.update_layout(title=dict(x=0.5))
fig.update_traces(legendgrouptitle=dict(text='<b>Condición de vida</b><br>'), marker=dict(line=dict(color='black',width=1.3)),
                 textinfo='percent', textfont=dict(color='black'))

fig.show()

**Interpretación:**

Como se ve en el gráfico el porcentaje de personas que consideran que cuentan con una condición de vida "buena" es más de la mitad de la población mientras que "muy malo" es casi insignificante.
 

## Base Suicidios

### 2. ¿Cuál es la proporción de casos de suicidio por grupo etareo y por género?

In [ ]:
# Creación del dataset
df = sc_df[sc_df['grupo_de_edad'] != 'Todos los grupos'].groupby(['grupo_etario', 'sexo'], as_index=False)[['casos']].sum()

# Creación de gráfica
fig = px.sunburst(df, path=['grupo_etario', 'sexo'], values='casos', color='grupo_etario', maxdepth=-1, color_discrete_sequence=px.colors.qualitative.T10,
                 hover_name='grupo_etario',  hover_data={'grupo_etario':False,}, title='<b>Proporción de suicidios por grupo etario y género</b>')

# Agregar detalles a gráfica
fig.update_layout(title_x=0.5)
fig.update_traces(textinfo='label+percent parent')
fig.update_traces(hovertemplate='<b>%{id}</b><br><b>Suicidios</b>: %{value}<extra></extra>')

fig.show()

**Interpretacion:**

En el gráfico se identifican las diferentes proporciones de cada grupo etario y en el siguiente nivel la proporción por cada sexo de este grupo, de resaltar la mayor propoción se da en los adultos (aunque sea el grupo etario con mayor rango de edad, es precupante la gran proporción) y también se observa que en todos los grupos etarios los hombres son los que más se suicidan.

### 3. ¿Cómo se comportan los casos de suicidio por grupo de edad cada año?

In [ ]:
# Creación del dataset
df = sc_df.loc[sc_df['grupo_de_edad'] != 'Todos los grupos'].groupby(['grupo_de_edad', 'year'], as_index=False)[['casos']].sum()

# Creación de gráfica
fig = px.bar(df,x='grupo_de_edad', y='casos', animation_frame='year', 
            title='<b>Casos de suicidio por grupo de edad cada año<b>', text='casos', template='plotly_white', range_y=[0, df['casos'].max()+20],
            color_discrete_sequence=['#EA380C'], labels={'casos':'<b>Casos</b>', 'grupo_de_edad':'<b>Grupo de edad</b>', 'year':'<b>Año<b>'},
            hover_data={'grupo_de_edad':False, 'casos':False, 'year':False}, hover_name='year')

# Agregar detalles a gráfica
fig.update_layout(title=dict(x=0.5))
fig.update_traces(textposition='outside')

for i in fig.frames:
    i['data'][0]['textposition'] = 'outside'
    
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500
    
fig.show()

**Interpretación:**

Como se observa en los gráficos, a través de los años se puede ver que los grupos de edad de 20 a 24 y 25 a 29 son los que más tienen casos de suicidio, también se aprecia que en el año 2020 (año donde aparece el COVID-19) hay una disminución considerable en los casos de cada grupo de edad.

### 4. ¿Cómo es la evolución anual de la tasa de suicidios por cien mil habitantes por cada localidad?

In [ ]:
# Creación del dataset
df = sc_df[(sc_df['grupo_de_edad'] == 'Todos los grupos') & (sc_df['localidad'] != 'distrito')].groupby(['year', 'localidad'], as_index=False)[['poblacion', 'tasa_por_100000']].agg('sum', 'mean')

# Creación de gráfica
fig = px.line(df, x='year', y='tasa_por_100000', facet_col='localidad', facet_col_wrap=5, markers=True,
              title='<b>Evolución anual de la tasa de suicidios por cada 100k habitantes por localidad<b>', template='plotly_white',
              labels={'year':'<b>Año</b>', 'tasa_por_100000':'<b>Tasa por<br>100K habs</b>'}, color_discrete_sequence=['lightskyblue'], hover_name='localidad',
              range_y=[-2, (df['tasa_por_100000'].max() + 5)])

# Agregar detalles a gráfica
fig.for_each_annotation(lambda a: a.update(text=a.text.title().split("=")[-1]))
fig.for_each_trace(lambda x: x.update(hovertemplate='<b>%{hovertext}</b><br><b>Año</b>=%{x}<br><b>Tasa</b>=%{y:.3}<extra></extra>')) 
fig.update_layout(title=dict(x=0.5))
fig.update_xaxes(showgrid=False, showline=False, fixedrange=True, nticks=df['year'].nunique()+1)

fig.show()

**Interpretación:**

En los gráficos de cada una de las localidades se puede ver la tasa de suicidios por cada 100.000 habitantes a lo largo de los años que en la mayoría tienen un comportamiento similar.

### 5. ¿Cuál es la evolución de los casos anuales de suicidio?

In [ ]:
# Creación del dataset
df = sc_df.query("localidad == 'distrito' & grupo_de_edad == 'Todos los grupos'").groupby('year', as_index=False)['casos'].sum()
df['acum'] = df['casos'].cumsum()

# Creación de gráfica
fig = px.line(df, x='year', y='casos', markers=True, template='plotly_white', text='casos',
             labels={'casos':'<b>Casos de suicidio</b>', 'year':'<b>Año</b>'}, title='<b>Casos anuales de suicidio<b>',
             color_discrete_sequence=['rgba(30, 124, 55, 0.55)'], hover_data={'year':False, 'casos':False})

# Agregar detalles a gráfica
fig.update_layout(title={'x':0.5})
fig.update_traces(textposition='top left', textfont={'size':16})
fig.update_xaxes(showgrid=False, fixedrange=True)
fig.update_yaxes(showgrid=False)
fig.show()

**Interpretación:**

En este gráfico se muestra la evolución de los casos anuales de suicidio y se observa que había una tendencia creciente y en el 2019 fue el máximo de casos, pero a partir de este año se ve una tendencia a disminuir a pesar de la llegada de la pandemia del COVID-19.

### 6. ¿Cuáles son los años que más aportan a la cantidad de suicidios total?

In [ ]:
# Creación del dataset
df = sc_df.query("grupo_de_edad == 'Todos los grupos' & localidad == 'distrito'").groupby('year', as_index=False)[['casos']].sum().sort_values('casos', ascending=False)
df['porcentaje'] = df[['casos']].apply(lambda x: x.cumsum()/x.sum())
df['year'] = df['year'].astype('string')

# Creación de gráfica
fig = go.Figure([go.Bar(x=df['year'], y=df['casos'], yaxis='y1', name='Suicidios', marker={'color':'rgba(182, 131, 227, 0.88)'}),
                 go.Scatter(x=df['year'], y=df['porcentaje'], yaxis='y2', name='Porcentaje', hovertemplate='%{y:.1%}', marker={'color': '#000000'})])

# Agregar detalles a gráfica
fig.update_layout(template='plotly_white', showlegend=False, hovermode='x', bargap=.3,
                  title={'text': '<b>Pareto suicidios por año<b>', 'x': .5}, 
                  yaxis={'title': '<b>Suicidios</b>', 'showgrid':False, 'fixedrange':True}, xaxis={'fixedrange':True},
                  yaxis2={'fixedrange':True, 'showgrid':False, 'rangemode': "tozero", 'overlaying': 'y', 'position': 1, 'side': 'right', 'title': '<b>Porcentaje</b>', 'tickvals': np.arange(0, 1.1, .2), 'tickmode': 'array', 'ticktext': [str(i) + '%' for i in range(0, 101, 20)]})

fig.show()

**Interpretación:**

Se realiza un diagrama de Pareto para analizar el comportamiento de la cantidad de suicidios observando los años en los que hubieron más casos y el porcentaje acumulado que aporta cada año al total. Se aprecia que las columnas tienen tamaños similares y hay una acumulación de los casos constantes, indicando así que cuando llegó la pandemia del COVID-19 no afectó en gran medida los casos de suicidio, en cambio disminuyeron a comparación de los otros años.

## Base Violencia Intrafamiliar

### 7. ¿Cuál es la evolución de los casos de violencia de cada tipo por cada años?

In [ ]:
# Creación del dataset
df = vi_df[(vi_df['localidad'] != 'distrito') & (vi_df['sexo'] != 'Total general') & (vi_df['tipo_de_violencia'] != 'Intrafamiliar')]
df = df.groupby(['tipo_de_violencia', 'year'], as_index=False)[['no_casos']].sum()
df['tipo_de_violencia'].unique()

# Creación de gráfica
fig = px.line(df, x='year', y='no_casos', color='tipo_de_violencia',
            labels={'no_casos':'<b>Casos<b>', 'tipo_de_violencia': '<b>Tipo de violencia<b>', 'year':'<b>Año<b>'}, 
            template='plotly_white',  title='<b>Evolución de casos por cada tipo de violencia a través de los años<b>',
            markers=True, color_discrete_sequence=px.colors.qualitative.Bold)

# Agregar detalles a gráfica
fig.update_layout(title=dict(x=0.5), xaxis={'showgrid':False})

fig.show()

**Interpretación:**

En el gráfico se presentan los casos de cada tipo de violencia y se evidencia que la violencia Emocional es la que más se presenta cada año, pero los valores de cada tipo se presentan relativamente constantes y similares cada año, pero a partir del año 2020 se empieza a ver un decrecimiento y en el 2021 se ve un descenso considerable de la cantidad total de casos de violencia intrafamiliar.

### 8. ¿Cuál es el porcentaje de los casos de cada tipo de violencia en la ciudad de Bogotá?

In [ ]:
# Creación del dataset
df = vi_df.query("sexo == 'Total general' & localidad == 'distrito' & tipo_de_violencia != 'Intrafamiliar'").groupby('tipo_de_violencia', as_index=False)[['no_casos']].sum()

# Creación de gráfica
fig = px.pie(df, values='no_casos', names='tipo_de_violencia', title='<b>Porcentaje de tipos de violencia en Bogotá</b>',
            color_discrete_sequence=px.colors.qualitative.Vivid, hover_name='tipo_de_violencia', hole=0.5)

# Agregar detalles a gráfica
fig.update_traces(legendgrouptitle={'text':'<b>Tipo de violencia</b><br>'}, marker={'line':{'color':'white', 'width':1.5}},
                 textfont={'color':'white'}, hovertemplate='<b>%{hovertext}</b><br>Casos:%{value:.2s}<extra></extra>',
                 textposition='inside')
fig.update_layout(title=dict(x=0.5), legend={'orientation':'h', 'title':{'side':'top left'}},
                 annotations=[{'text':f'<b>Total<br>{int(df["no_casos"].sum())}</b>', 'showarrow':False, 'font':{'size':25}}])

fig.show()

**Interpretación:**

En este gráfico se presenta el procentaje total de cada tipo de violencia, en donde la Emocional representa casi la mitad de los casos y entre el Abandono y la Económica solo aportan el 5% de la totalidad. También se tiene el total de casos presentados en los años que se tiene información para entrar en contexto.

### 9. ¿Cómo es la evolución de la violencia intrafamiliar sufrida por género?


In [ ]:
# Creación del dataset
df = vi_df[vi_df['sexo'] != 'Total general'].groupby(['sexo', 'year'], as_index=False)[['no_casos']].sum()
df['year'] = df['year'].astype('category')

# Creación de gráfica
fig = px.line(df, x='year', y='no_casos', color='sexo', markers=True, symbol='sexo', template='plotly_white',
            labels={'no_casos':'<b>Casos</b>', 'year':'<b>Año</b>', 'sexo':'Genero'}, title='<b>Violencia intrafamiliar sufrida por género</b>', 
            hover_name='year', hover_data={'sexo':False, 'year':False})

# Agregar detalles a gráfica
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><b>Casos</b>=%{y}<extra></extra>')
fig.update_layout(xaxis=dict(showline=False, showgrid=False), title=dict(x=0.5))
fig.update_xaxes(fixedrange=True)

fig.show()

**Interpretación:**

Como se observa en la gráfica todos los años hay más casos de violencia intrafamiliar en las mujeres, cabe resaltar que en el año 2020 los casos en ambos sexos empiezan a bajar y se ve una disminución mayor en las mujeres


## Relación entre Bases

### 10. ¿Cuál es la relación entre casos de suicidio y casos de violencia por localidad por año?.

In [ ]:
# Creación del dataset
_1 = sc_df.query("localidad != 'distrito' & grupo_de_edad == 'Todos los grupos'").groupby(['year', 'localidad'], as_index=False)['casos'].sum()

_2 = vi_df.query("localidad != 'distrito' & sexo == 'Total general' & tipo_de_violencia != 'Intrafamiliar'").groupby(['year', 'localidad'], as_index=False)[['no_casos', 'poblacion']].agg('mean', lambda x: set(x))

df = pd.merge(_1, _2, on=['localidad', 'year'], how='inner').rename(columns={'casos':'suicidios', 'no_casos':'violencia'})

# Creación de gráfica
fig = px.scatter(df, y='suicidios', x='violencia', color='localidad', animation_frame='year', size='poblacion',
                color_discrete_sequence=px.colors.qualitative.Dark2, labels={'localidad':'<b>Localidad</b><br>', 'suicidios':'<b>Casos de suicidio</b>', 'violencia':'<b>Casos de violencia</b>', 'year':'Año'}, 
                template='plotly_white', hover_name='localidad', hover_data={'localidad':False, 'year':False, 'violencia':True, 'suicidios':True},
                range_x=[-1, df['violencia'].max() + 80], range_y=[0.3, df['suicidios'].max() + 3], trendline='ols', trendline_scope='overall',
                trendline_color_override='black')

# Agregado de detalles y línea de tendencia a todos los frames de la gráfica
for i,j in zip(df['year'].unique(), fig.frames):
    df = df.copy()
    df_1 = df[df['year'] == i]
    j.data = px.scatter(df_1, y='suicidios', x='violencia', color='localidad', size='poblacion',
                        color_discrete_sequence=px.colors.qualitative.Dark2, labels={'localidad':'<b>Localidad</b><br>', 'suicidios':'<b>Casos de suicidio</b>', 'violencia':'<b>Casos de violencia</b>', 'year':'Año'}, 
                        template='plotly_white', hover_name='localidad', hover_data={'localidad':False, 'year':False, 'violencia':True, 'suicidios':True},
                        trendline='ols', trendline_scope='overall', trendline_color_override='black').data
    j.layout['annotations'] = [{'text':r"$\rho={}$".format(round(df.groupby('year').corr().loc[(i,'suicidios'), 'violencia'], 2)), 'showarrow':False,
                                'font':{'size':13}, 'x':110, 'y':55}]

# Agregar detalles a gráfica
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout(annotations=[{'text':r"$\rho={}$".format(round(df.groupby('year').corr().loc[(2015,'suicidios'), 'violencia'], 2)), 'showarrow':False,
                                'font':{'size':13}, 'x':110, 'y':55}])

fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>Casos de violencia=%{x:.3s}<br>Casos de suicidio=%{y}<br>Población=%{marker.size:}<extra></extra>')
fig.data[-1]['showlegend'] = False
fig.data[-1]['hovertemplate'] = '<extra></extra>'

for x in fig.frames:
    for i in x.data:
        i['hovertemplate'] = '<b>%{hovertext}</b><br><br>Casos de violencia=%{x:.3s}<br>Casos de suicidio=%{y}<br>Población=%{marker.size:}<extra></extra>'

fig.layout.sliders[0]['currentvalue']['prefix'] = '<b>Año</b>:'
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

for i in fig.frames:
    i.data[-1]['showlegend'] = False
    i.data[-1]['hovertemplate'] = '<extra></extra>'

fig.show()

**Interpretación:**

En este gráfico de correlación se muestra la relación que hay entre los casos de suicidio y los casos de violencia por cada año y localidad, se observa que sí hay cierta relación y que a mayor cantidad de casos de violencia intrafamiliar suele haber más casos de suicidio, constatado por el coeficiente de Pearson que es cercano a 1. También se observa que el ancho de los círculos representa el total de la población, que debe ser tenido en cuenta ya que podría afectar el total de casos.

### 11. ¿Cuál es la relación del puntaje de percepción de vida mala y muy mala con la tasa de casos de violencia por cien mil habitantes por localidad en el año 2017?

In [ ]:
# Creación del dataset
_1 = vi_df.query("year == 2017 & localidad != 'distrito' & sexo == 'Total general' & tipo_de_violencia != 'Intrafamiliar'").groupby('localidad', as_index=False)[['tasa_por_100000', 'poblacion']].agg('sum', lambda x: set(x))

_2 = cv_df[cv_df['condiciones_de_vida'].isin(['malo', 'muy malo'])].groupby('localidad', as_index=False)['porcentaje'].sum()

df = pd.merge(_1, _2, on='localidad', how='inner')

# Creación de gráfica
fig = px.scatter(df, y='porcentaje', x='tasa_por_100000', color='localidad', size='poblacion', template='plotly_white',
                labels={'tasa_por_100000':'<b>Tasa de casos de violencia por cada 100K habitantes</b>', 'porcentaje':'<b>Puntaje de percepción de vida</b>', 'localidad':'<b>Localidad</b><br>'},
                hover_name='localidad', hover_data={'localidad':False}, trendline='ols', trendline_scope='overall',
                trendline_color_override='black', color_discrete_sequence=px.colors.qualitative.Alphabet, title='<b>Relación entre el puntaje de percepción de vida mala y muy mala<br>con la tasa de casos de violencia por cada 100K habitantes</b>')

# Agregar detalles a gráfica
fig.update_layout(title={'x':0.5}, annotations=[{'text':r"$\rho={}$".format(round(df.corr().loc['porcentaje', 'tasa_por_100000'], 2)), 'showarrow':False,
                                                 'font':{'size':13}, 'x':1500, 'y':2.7}])
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>Tasa=%{x:.1f}<br>Puntaje=%{y}%<br>Población=%{marker.size}<extra></extra>')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


fig.data[-1]['showlegend'] = False
fig.data[-1]['hovertemplate'] = '<extra></extra>'


fig.show()

**Interpretación:**

En este gráfico se presenta la correlación entre el puntaje acumulado de la percepción de vida Mala y Muy mala y la tasa de casos por cada 100.000 habitantes y se observa que sí hay cierta correlación ya que la mayoría de los puntos están cerca a la línea, aunque el coeficiente de Pearson es igual a 0.46 lo cual indicaría que la correlación no es tan fuerte. También de resaltar que al usar la tasa por cada 100.000 habitantes, los resultados se encuentran normalizados de cierta forma y esto hace que las localidades con mayor población no tengan la mayor cantidad de casos y afecte la correlación.